# Training a Neural Network with PyTorch

In [1]:
import torch
import torch.nn as nn #Base class for all neural network modules.
import matplotlib.pyplot as plt
import pandas as pd

## Load data

In [2]:
# in case of colab notebook
from google.colab import drive
drive.mount('/content/drive')

cdir = '' #path to your directory

Mounted at /content/drive


In [3]:
data = pd.read_csv(cdir + 'california_housing_train.csv')
data.describe()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
count,17000.000000,17000.000000,17000.000000,17000.000000,17000.000000,17000.000000,17000.000000,17000.000000,17000.000000
mean,-119.562108,35.625225,28.589353,2643.664412,539.410824,1429.573941,501.221941,3.883578,207300.912353
std,2.005166,2.137340,12.586937,2179.947071,421.499452,1147.852959,384.520841,1.908157,115983.764387
min,-124.350000,32.540000,1.000000,2.000000,1.000000,3.000000,1.000000,0.499900,14999.000000
25%,-121.790000,33.930000,18.000000,1462.000000,297.000000,790.000000,282.000000,2.566375,119400.000000
50%,-118.490000,34.250000,29.000000,2127.000000,434.000000,1167.000000,409.000000,3.544600,180400.000000
75%,-118.000000,37.720000,37.000000,3151.250000,648.250000,1721.000000,605.250000,4.767000,265000.000000
max,-114.310000,41.950000,52.000000,37937.000000,6445.000000,35682.000000,6082.000000,15.000100,500001.000000


In [4]:
#Fill NaN values
data = data.fillna(0)
#Normalize values
data = (data-data.mean())/data.std()
#Separate features and targets
x_df = pd.DataFrame(data, columns=data.columns[:-1])
y_df = pd.DataFrame(data, columns=[data.columns[-1]]) #target = 'median_house_value'
#Save in tensors
x = torch.tensor(x_df.values, dtype=torch.float)
y = torch.tensor(y_df.values, dtype=torch.float)

print(f"x shape: {x.shape}")
print(f"y shape: {y.shape}")

x shape: torch.Size([17000, 8])
y shape: torch.Size([17000, 1])


## Create a Neural Network

In [5]:
class Net(nn.Module):
    def __init__(self, D_in, H1, H2, H3, D_out):
        super(Net, self).__init__()

        self.linear1 = nn.Linear(D_in, H1) # You can also try include activation functions to check how your model will behave
        self.linear2 = nn.Linear(H1, H2)
        self.linear3 = nn.Linear(H2, H3)
        self.linear4 = nn.Linear(H3, D_out)

    def forward(self, x):
        h1 = self.linear1(x)
        h2 = self.linear2(h1)
        h3 = self.linear3(h2)
        out = self.linear4(h3)
        return out

In [6]:
#Define layer sizes
D_in = x.shape[1] #size of the input sample
H1 = 128
H2 = 64
H3 = 32
D_out = 1

#Define Hyperparameters
learning_rate = 1e-4 # You can also experiment with different learning rates

#Initialise model, loss, optimizer
model = Net(D_in, H1, H2, H3, D_out)
loss_func = nn.MSELoss(reduction='sum') # You can also try BCELoss and BCEWithLogitsLoss
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate) # You can also try Adam and AdamW

#Initialise dataloader
dataset = torch.utils.data.TensorDataset(x, y) #class to represent the data as list of tensors. x=input_features, y=labels
dataloader = torch.utils.data.DataLoader(dataset, batch_size=64, shuffle=True)

In [7]:
model

Net(
  (linear1): Linear(in_features=8, out_features=128, bias=True)
  (linear2): Linear(in_features=128, out_features=64, bias=True)
  (linear3): Linear(in_features=64, out_features=32, bias=True)
  (linear4): Linear(in_features=32, out_features=1, bias=True)
)

## Train Network

In [8]:
for epoch in range(5):
  batch_losses = []

  for x_batch, y_batch in dataloader:
    y_pred = model(x_batch)

    loss = loss_func(y_pred, y_batch)
    batch_losses.append(loss.item())
    # print('y_pred=', y_pred[0])
    #Delete previously stored gradients
    optimizer.zero_grad()
    #Perform backpropagation starting from the loss calculated in this epoch
    loss.backward()
    #Update model's weights based on the gradients calculated during backprop
    optimizer.step()

  print(f"Epoch {epoch:3}: Loss = {sum(batch_losses)/len(dataloader):.5f}")


Epoch   0: Loss = 29.48857
Epoch   1: Loss = 23.76013
Epoch   2: Loss = 23.27231
Epoch   3: Loss = 23.23699
Epoch   4: Loss = 23.20877


##Optimization

Optuna framework

Optuna Concepts
- Objective Function: This is the function that defines the machine learning task you're trying to optimize. In our case, it includes the code for training and validating a neural network model. Optuna will run this function multiple times with different hyperparameter settings.

- Trial: A trial is a single execution of the objective function using a specific set of hyperparameters. Each trial evaluates how well that particular configuration performs.

- Study: A study is a collection of trials. Optuna runs several trials as part of a study to explore different hyperparameter combinations. After all trials are complete, the study helps identify the best-performing configuration—i.e., the one that minimizes (or maximizes) the objective function.

- Parameter: These are the values we are trying to optimize—like learning rate, optimizer type, or layer sizes. Each trial uses a different set of parameters. The goal is to find the combination that results in the best performance of the model.

In [9]:
# !pip install optuna
import optuna

#Optuna sample

def suggest_hyperparameters(trial): #function to include the suggested hyperparameters
    # Experimenting with different optimizers
    optimizer_name = trial.suggest_categorical("optimizer_name", ["Adam", "AdamW"])
    return optimizer_name

#Optuna sample
def objective(trial):
    optimizer_name = suggest_hyperparameters(trial)
    #Define layer sizes
    D_in = x.shape[1]
    H1 = 128 #size of the input sample
    H2 = 64
    H3 = 32
    D_out = 1

    #Define Hyperparameters
    learning_rate = 1e-4 # You can also experiment with different learning rates

    #Initialise model, loss, optimizer
    model = Net(D_in, H1, H2, H3, D_out)
    loss_func = nn.MSELoss(reduction='sum') # You can also try BCELoss and BCEWithLogitsLoss
    if optimizer_name == "Adam":
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    elif optimizer_name == "AdamW":
        optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
    # elif optimizer_name == "SGD":
    #     optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

    #Initialise dataloader
    dataset = torch.utils.data.TensorDataset(x, y) #class to represent the data as list of tensors. x=input_features, y=labels
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=64, shuffle=True)
    for epoch in range(5):
        batch_losses = []

        for x_batch, y_batch in dataloader:
            y_pred = model(x_batch)

            loss = loss_func(y_pred, y_batch)
            batch_losses.append(loss.item())
            # print('y_pred=', y_pred[0])
            #Delete previously stored gradients
            optimizer.zero_grad()
            #Perform backpropagation starting from the loss calculated in this epoch
            loss.backward()
            #Update model's weights based on the gradients calculated during backprop
            optimizer.step()

        print(f"Epoch {epoch:3}: Loss = {sum(batch_losses)/len(dataloader):.5f}")
        epoch_loss = sum(batch_losses)/len(dataloader)
    return epoch_loss

optuna_sample = optuna.create_study(direction = 'minimize' , study_name = 'lr-minim-sample')
optuna_sample.optimize(objective, n_trials = 50) #the first parameter is the function that we want to optimise
print('numbers of the finished trials:' , len(optuna_sample.trials))
print('the best params:' , optuna_sample.best_trial.params)
print('the best value:' , optuna_sample.best_value)

# Best number of trials appears to be 50 , as it gave me the best hyperparameters for my model.

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.6 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 18.5 MB/s eta 0:00:00


[I 2025-04-02 06:27:39,012] A new study created in memory with name: lr-minim-sample


Epoch   0: Loss = 31.29959
Epoch   1: Loss = 23.51294
Epoch   2: Loss = 23.21660
Epoch   3: Loss = 23.17496


[I 2025-04-02 06:27:42,065] Trial 0 finished with value: 23.21031472198945 and parameters: {'optimizer_name': 'AdamW'}. Best is trial 0 with value: 23.21031472198945.


Epoch   4: Loss = 23.21031
Epoch   0: Loss = 34.60865
Epoch   1: Loss = 24.01241
Epoch   2: Loss = 23.20606
Epoch   3: Loss = 23.16348


...

Epoch   4: Loss = 23.05640
Epoch   0: Loss = 32.43928
Epoch   1: Loss = 23.73180
Epoch   2: Loss = 23.19591
Epoch   3: Loss = 23.14503


[I 2025-04-02 06:29:57,017] Trial 48 finished with value: 23.146095713278406 and parameters: {'optimizer_name': 'AdamW'}. Best is trial 16 with value: 23.040539906437235.


Epoch   4: Loss = 23.14610
Epoch   0: Loss = 32.32622
Epoch   1: Loss = 23.83744
Epoch   2: Loss = 23.28816
Epoch   3: Loss = 23.17083


[I 2025-04-02 06:29:59,593] Trial 49 finished with value: 23.129662976229103 and parameters: {'optimizer_name': 'AdamW'}. Best is trial 16 with value: 23.040539906437235.


Epoch   4: Loss = 23.12966
numbers of the finished trials: 50
the best params: {'optimizer_name': 'AdamW'}
the best value: 23.040539906437235
